# FrozenQubits:   Boosting Fidelity of QAOA by Skipping Hotspot Nodes

FrozenQubits is a novel quantum divide-and-conquer approach that leverages the insight about most real-world applications having power-law graphs to boost the fidelity of QAOA applications. 
In FrozenQubits, we freeze some of the nodes with the highest degree of connectivity. 
This drops the edges connected to these nodes resulting in a circuit with fewer CNOTs and lower depth. 

To read the full paper, please visit: (https://arxiv.org/abs/2210.17037)(https://arxiv.org/abs/2210.17037)

This tutorial includes some example codes showing how to apply FrozenQubits to QAOA applications and compare its results with the standard QAOA. 


## Requirements

The following packages are needed to run the sample codes:

- numpy (pip install numpy)
- qiskit (pip install qiskit)
- networkx (pip install networkx)
- dimod (pip install dimod)

In [46]:
from helper import *
from helper_qaoa import *
from  helper_FrozenQubits import *
from Evaluation import *

import random
import colorama
from colorama import Fore

from os import *
from qiskit import *

## Specifying Benchmark Problem

This repository includes three types of problems:
- SK: Sherrington—kirkpatrick (SK) model or fully-connected graphs 
- k_regular: 3-Regular graphs
- ba: Barabasi—Albert (BA) or power-law graphs 


In addition to the benchmark type, we should specify the number of qubits (or nodes of the problem graph), denoted by $n \in \{4, 5, \dots, 24\}$ and the value of $k.$
- In "k_regular" graphs, $k$ specifies the degree of the nodes, and only $k=3$ examples are available in this repository. 
- In BA graphs, $k$ specifies the preferential attachment $(d_{BA}),$ and $k=1,2$ and 3 benchmarks are included in this repository.


In [68]:
benchmark_type=['sk', 'k_regular', 'ba'][2]

# Number of nodes/qubits
n=8
print(f'Benchmark: {benchmark_type} graph with {n} nodes')

k=1
# in BA graphs, k=1,2 or 3
#for k_regular, only k=3 is available 
# Also, for k-Regular graphs, 1<k<n and k*n must be an even number

# We only have data for P=1 layers of QAOA 
num_layers=1
benchmark_id=""

try:
    benchmark_id=get_benchmark_id(benchmark_type, n=n, k=k)+f'^P={num_layers}'
except:
    pass
    # in k-regular graphs, 1<k<n, and n*k ust be an even number

Benchmark: ba graph with 8 nodes


## Loading Benchmark Problem

In [48]:
# baseline files are in the following folder:
qaoa_path=f'experiments/qaoa/{benchmark_type}/gridsearch_100/'

# Results for the ideal simulator on in the "ideal" subfolder 

print("printing current directory: %s", cwd)
qaoa_ideal_filename=f'{qaoa_path}ideal/{benchmark_id}.pkl'
qaoa_ideal_obj=load_pickle(qaoa_ideal_filename)
#qaoa_ideal_obj is a dict object 

# all benchmark files include h, J and offset that defines the Ising Hamiltonian 
# h denotes linear coefficients
# J denotes quadratic coefficients
# offset is the constant bias
h=qaoa_ideal_obj['h']
J=qaoa_ideal_obj['J']
offset=qaoa_ideal_obj['offset']

print(f'{benchmark_type} benchmark with {n} nodes loaded')
print('h=', h)
print('J=', J)
print('offset=', offset)

# for each benchmark, we have found the optimum solution(s) by Brute force approach
cost_best=qaoa_ideal_obj['cost_best']
optimum_solutions=qaoa_ideal_obj['optimum_solutions']

print('--------')
print('Minimum cost value=', cost_best)
print('Optimum solutions:', optimum_solutions)

NameError: name 'benchmark_type' is not defined

## Creating Parametric QAOA Circuit 

In [35]:
# the "helper_qaoa.py" (which has been published with this tutorial) 
# includes "pqc_QAOA" module that creates a parametric QAOA circuit 
# for an Ising Hamiltonian, defined with h and J
_out=pqc_QAOA(J=J, h=h, num_layers=num_layers)
qaoa_circ=_out['qc']

print(f'QAOA circuit with P={num_layers} layers created')
print('Number of qubits:', qaoa_circ.num_qubits)
print('Depth:', qaoa_circ.depth())
print('CNOT count:', qaoa_circ.count_ops()['cx'])

QAOA circuit with P=1 layers created
Number of qubits: 8
Depth: 24
CNOT count: 16


In [36]:
#and here is the circuit layout 
print(qaoa_circ)

     ┌───┐                                                                    »
q_0: ┤ H ├──■────────────────────■────■────────────────────────────────────■──»
     ├───┤┌─┴─┐┌──────────────┐┌─┴─┐  │                                    │  »
q_1: ┤ H ├┤ X ├┤ Rz(-2.0*g_1) ├┤ X ├──┼─────────■──────────────────────────┼──»
     ├───┤└───┘└──────────────┘└───┘  │       ┌─┴─┐      ┌──────────────┐  │  »
q_2: ┤ H ├────────────────────────────┼───────┤ X ├──────┤ Rz(-2.0*g_1) ├──┼──»
     ├───┤                            │       └───┘      └──────────────┘  │  »
q_3: ┤ H ├────────────────────────────┼────────────────────────────────────┼──»
     ├───┤                            │                                    │  »
q_4: ┤ H ├────────────────────────────┼────────────────────────────────────┼──»
     ├───┤                            │                                    │  »
q_5: ┤ H ├────────────────────────────┼────────────────────────────────────┼──»
     ├───┤                            │ 

## Training Circuit Parameters $(\gamma$ and $\beta)$

In [37]:
# We must use a classical optimizer to find the optimum values of circuit parameters, gamma and beta 

# We already have searched on the grid of 100x100 to find the optimum parameter values
x_best=qaoa_ideal_obj['x_best']
_gamma=x_best[0]
_beta=x_best[1]
print('Optimum circuit parameters (num_layers=1):')
print('\t γ=', _gamma)
print('\t β=', _beta)

EV_best=qaoa_ideal_obj['EV_best']
print('EV_best (ideal)=', EV_best)
print('Approximation Ratio (ideal)=', EV_best/cost_best)

NameError: name 'qaoa_ideal_obj' is not defined

## Running the QAOA Circuit on a Real Quantum Hardware

In [ ]:
# Now, we can replace circuit parameters with optimum values of gamma and beta
# compile it to a real quantum computer,
# and run it on a real device
qaoa_circ_binded=bind_QAOA(qaoa_circ, _out['params'], gamma=_gamma, beta=_beta)                        

# we already have provided the qasm file of the circuit with optimum values
qaoa_ideal_qasm_filename=f'{qaoa_path}ideal/{benchmark_id}.qasm'
circ=qiskit.circuit.QuantumCircuit.from_qasm_file(qaoa_ideal_qasm_filename)

# For all benchmarks, we have run the circuit with optimum parameters 
# on eight different quantum computers by IBM
machine_name='ibmq_montreal'
# results from running the circuit with optimum parameter values are in the folders
# with IBM machine names
# results for 8 different IBM machines are available for all benchmarks

qaoa_real_filename=f'{qaoa_path}{machine_name}/{benchmark_id}.pkl'
print(f'qaoa real file name: {qaoa_real_filename}')
qaoa_real_obj=load_pickle(qaoa_real_filename)
# qaoa_real_obj has a structure that is similar to qaoa_ideal_obj

EV_real_best=qaoa_real_obj['EV_best']
print(f'EV_best ({machine_name})=', EV_real_best)
print(f'Approximation Ratio ({machine_name}) =', EV_real_best/cost_best)

## Applying FrozenQubits

In FrozenQubits, we must specify the number of qubits to freeze, denoted by $m.$
We have results for $m=1$ and 2.

## Identifying and Removing Hotspot nodes

In [ ]:
m=1
# number of qubits to freeze
# results for m=1 and 2 have been included in this repository 

G=nx.Graph()
G.add_edges_from(list(J.keys()))
G.add_nodes_from(list(h.keys()))
print('Edges: ', G.edges)

list_of_halting_qubits=[]
for i in range(m):
    G, list_of_halting_qubits=drop_hotspot_node(G, list_of_fixed_vars=list_of_halting_qubits, verbosity=0)                                                        

print('Nodes to drop:', list_of_halting_qubits)

## Freeze Qubits and Create Sub-Problems

Freezing $m$ qubits will result in $2^m$ sub-problems.

Note: Each sub-problem has its own $h$, $J$ and offset

In [ ]:
sub_Ising_list=halt_qubits(J=J, h=h, offset=offset, halting_list=list_of_halting_qubits)                

print(f'For m={m}, {len(sub_Ising_list)} sub-problems created')



In [ ]:
# exploring the sub-problems 

for sub_index in range(len(sub_Ising_list)):
    print('------Sub-problem number:', sub_index )
    sub_problem=sub_Ising_list[sub_index ]    
    print('h:', sub_problem['h'])
    print('J', sub_problem['J'])
    print('offset:', sub_problem['offset'])
    # most quantum devices do not accept arbitrary qubit numbers, 
    # so we encode node names 
    print('encoding node names:', sub_problem['encoding'])
    print('FrozenQubits values:', sub_problem['halting_vals'])
    print()

## Creating QAOA Circuits for Sub-Problems

In [ ]:
# For each sub-problem, we can create a QAOA circuit 
#  Note: each sub-problem includes its own h, J and offset

#Example: Creating QAOA circuit 
# for the first sub-problem
sub_problem=sub_Ising_list[0]    
_out_i=pqc_QAOA(J=sub_problem['J'], h=sub_problem['h'], num_layers=num_layers)
qc_i=_out_i['qc']

print('QAOA circuit for sub-problem created')
print('Number of Qubits:', qc_i.num_qubits)
print('Circuit Depth:', qc_i.depth())
print('CNOT Count:', qc_i.count_ops().get('cx', []))

## Training Sub-QAOA Circuitsof 

In [ ]:
# we have included results of training all sub-problems in this repository 

# FrozenQubits files are in the following folder:
FQ_path=f'experiments/frozenqubits_full/{benchmark_type}/gridsearch_100/'

sub_index=0
# sub_index can be between 0 to 2^m -1
FQ_id=get_benchmark_id(benchmark_type, n=n, k=k)+f'^M={m}_{sub_index}^P={num_layers}'

# Results for the ideal simulator is in in the "ideal" subfolder 
FQ_ideal_filename_i=f'{FQ_path}ideal/{FQ_id}.pkl'
FQ_ideal_obj_i=load_pickle(FQ_ideal_filename_i)
#FQ_ideal_obj is a dict object 

print(f'm={m} | sub-index={sub_index}')
print(f'EV_best (ideal)=', FQ_ideal_obj_i['EV_best'])
print(f'Approximation Ratio (ideal)= ', FQ_ideal_obj_i['EV_best']/FQ_ideal_obj_i['cost_best'])

## Running Sub-Problems on a Real Quantum Hardware

In [ ]:
# Results from running all sub-problems on eight different IBM machines are also included 

FQ_real_filename_i=f'{FQ_path}{machine_name}/{FQ_id}.pkl'
FQ_real_obj_i=load_pickle(FQ_real_filename_i)
#FQ_real_obj_iis a dict object 

print('Machine:', machine_name)
print(f'EV_best ({machine_name})=', FQ_real_obj_i['EV_best'])
print(f'Approximation Ratio ({machine_name})=', FQ_real_obj_i['EV_best']/FQ_real_obj_i['cost_best'])

## Comparing Results

In [ ]:
print('Benchmark Type:', benchmark_type)
print('Number of problem variables:', n)
print(f'k={k} (for SK model, k is ignored)')

print()
print('Baseline (standard QAOA)')
print(f'\t EV (ideal):', qaoa_ideal_obj['EV_best'])
print(f'\t EV ({machine_name}):', qaoa_real_obj['EV_best'])
print(f'\t AR (ideal):', qaoa_ideal_obj['EV_best']/qaoa_ideal_obj['cost_best'])
print(f'\t AR ({machine_name}):', qaoa_real_obj['EV_best']/qaoa_ideal_obj['cost_best'])

print('--------------------------------')
print(f'FrozenQubits (m={m})')
for sub_index in range(2**m):
    print(f'  sub-problem ', sub_index)
    FQ_id=get_benchmark_id(benchmark_type, n=n, k=k)+f'^M={m}_{sub_index}^P={num_layers}'
    FQ_ideal_filename_i=f'{FQ_path}ideal/{FQ_id}.pkl'
    FQ_ideal_obj_i=load_pickle(FQ_ideal_filename_i)
    FQ_real_filename_i=f'{FQ_path}{machine_name}/{FQ_id}.pkl'
    FQ_real_obj_i=load_pickle(FQ_real_filename_i)
    print(f'\t EV (ideal):', FQ_ideal_obj_i['EV_best'])
    print(f'\t EV ({machine_name}):', FQ_real_obj_i['EV_best'])
    print(f'\t AR (ideal):', FQ_ideal_obj_i['EV_best']/FQ_ideal_obj_i['cost_best'])
    print(f'\t AR ({machine_name}):', FQ_real_obj_i['EV_best']/FQ_real_obj_i['cost_best'])
    

## Evaluating 2-regular Graphs
first we have to create a random 2-regular circuit with number of qubits between {4, 24} graph and then apllying the FrozenQubit of the newly created graph and as last step we compare the result


In [45]:
n = random.randint(4, 10)

print("running", n, "trials")

# running trials and getting CNOTs difference for all [4..24] circuits
results_2_regular = run_average_2_regular(n)

averages_2_regular = average_results(results_2_regular)
print("comparing average result on all circuits with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)\n")

# coloring can be removed
print(Fore.GREEN + "n, m_0, m_1, m_2")
print(Fore.CYAN)


print_matrix_as_csv(averages_2_regular)

running 8 trials
comparing average result on all circuits with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)

n, m_0, m_1, m_2

4, 8.0, 4.0, 0.0
5, 10.0, 6.0, 2.0
6, 12.0, 8.0, 4.0
7, 14.0, 10.0, 6.0
8, 16.0, 12.0, 8.0
9, 18.0, 14.0, 10.0
10, 20.0, 16.0, 12.0
11, 22.0, 18.0, 14.0
12, 24.0, 20.0, 16.0
13, 26.0, 22.0, 18.0
14, 28.0, 24.0, 20.0
15, 30.0, 26.0, 22.0
16, 32.0, 28.0, 24.0
17, 34.0, 30.0, 26.0
18, 36.0, 32.0, 28.0
19, 38.0, 34.0, 30.0
20, 40.0, 36.0, 32.0
21, 42.0, 38.0, 34.0
22, 44.0, 40.0, 36.0
23, 46.0, 42.0, 38.0
24, 48.0, 44.0, 40.0


## Comparing relative CNOTs count

In [44]:
print("printing relative CNOTs copyrison with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)\n")

# coloring can be removed
print(Fore.GREEN + "n, m_0, m_1, m_2")
print(Fore.CYAN)

print_matrix_as_csv(matrix_to_relative(averages_2_regular))

printing relative CNOTs copyrison with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)

n, m_0, m_1, m_2

4, 1.0, 0.5, 0.0
5, 1.0, 0.6, 0.2
6, 1.0, 0.6666666666666666, 0.3333333333333333
7, 1.0, 0.7142857142857143, 0.42857142857142855
8, 1.0, 0.75, 0.5
9, 1.0, 0.7777777777777778, 0.5555555555555556
10, 1.0, 0.8, 0.6
11, 1.0, 0.8181818181818182, 0.6363636363636364
12, 1.0, 0.8333333333333334, 0.6666666666666666
13, 1.0, 0.8461538461538461, 0.6923076923076923
14, 1.0, 0.8571428571428571, 0.7142857142857143
15, 1.0, 0.8666666666666667, 0.7333333333333333
16, 1.0, 0.875, 0.75
17, 1.0, 0.8823529411764706, 0.7647058823529411
18, 1.0, 0.8888888888888888, 0.7777777777777778
19, 1.0, 0.8947368421052632, 0.7894736842105263
20, 1.0, 0.9, 0.8
21, 1.0, 0.9047619047619048, 0.8095238095238095
22, 1.0, 0.9090909090909091, 0.8181818181818182
23, 1.0, 0.9130434782608695, 0.8260869565217391
24, 1.0, 0.9166666666666666, 0.8333333333333334


## Evaluating RGCG Graphs
first we have to create a RGCG circuit with number of qubits between {4, 24} graph and then apllying the FrozenQubit of the newly created graph and as last step we compare the result


In [49]:
n = random.randint(4, 10)

print("running", n, "trials")

# running trials and getting CNOTs difference for all [4..24] circuits with out any additional edges
results_random = run_average_random(n, 0)
averages_random = average_results(results_random)

print("comparing average result on all circuits with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)\n")
print(Fore.GREEN + "n, m_0, m_1, m_2")
print(Fore.CYAN)

print_matrix_as_csv(averages_random)

running 9 trials
comparing average result on all circuits with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)

n, m_0, m_1, m_2

4, 6.0, 1.7777777777777777, 0.0
5, 8.0, 3.5555555555555554, 0.4444444444444444
6, 10.0, 4.444444444444445, 1.5555555555555556
7, 12.0, 6.0, 2.2222222222222223
8, 14.0, 7.333333333333333, 3.5555555555555554
9, 16.0, 9.11111111111111, 4.888888888888889
10, 18.0, 10.666666666666666, 6.0
11, 20.0, 12.666666666666666, 7.555555555555555
12, 22.0, 14.88888888888889, 9.11111111111111
13, 24.0, 16.0, 10.444444444444445
14, 26.0, 19.11111111111111, 13.11111111111111
15, 28.0, 20.666666666666668, 14.666666666666666
16, 30.0, 22.22222222222222, 15.555555555555555
17, 32.0, 24.22222222222222, 18.22222222222222
18, 34.0, 25.77777777777778, 19.333333333333332
19, 36.0, 28.0, 21.555555555555557
20, 38.0, 30.88888888888889, 24.88888888888889
21, 40.0, 31.555555555555557, 25.11111111111111
22, 42.0, 32.888888888888886, 26.0
23, 44.0, 36.666666666666664

## Comparing relative CNOTs count

In [50]:
print("printing relative CNOTs copyrison with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)\n")

# coloring can be removed
print(Fore.GREEN + "n, m_0, m_1, m_2")
print(Fore.CYAN)


print_matrix_as_csv(matrix_to_relative(averages_random))

printing relative CNOTs copyrison with n = number of qubits, m_0 = FQ(m = 0), m_1 = FQ(m = 1), m_2 = FQ(m = 2)

n, m_0, m_1, m_2

4, 1.0, 0.2962962962962963, 0.0
5, 1.0, 0.4444444444444444, 0.05555555555555555
6, 1.0, 0.4444444444444445, 0.15555555555555556
7, 1.0, 0.5, 0.1851851851851852
8, 1.0, 0.5238095238095238, 0.25396825396825395
9, 1.0, 0.5694444444444444, 0.3055555555555556
10, 1.0, 0.5925925925925926, 0.3333333333333333
11, 1.0, 0.6333333333333333, 0.37777777777777777
12, 1.0, 0.6767676767676768, 0.41414141414141414
13, 1.0, 0.6666666666666666, 0.4351851851851852
14, 1.0, 0.735042735042735, 0.5042735042735043
15, 1.0, 0.7380952380952381, 0.5238095238095238
16, 1.0, 0.7407407407407407, 0.5185185185185185
17, 1.0, 0.7569444444444444, 0.5694444444444444
18, 1.0, 0.7581699346405228, 0.5686274509803921
19, 1.0, 0.7777777777777778, 0.5987654320987654
20, 1.0, 0.8128654970760234, 0.6549707602339181
21, 1.0, 0.788888888888889, 0.6277777777777778
22, 1.0, 0.783068783068783, 0.619047619

## Conclusion
FrozenQubits was introduced as an application-level software framework to improve the fidelity of Quantum Approximation Optimization Algorithm (QAOA), it works by freezing a qubit or more by substituting the two possibilities in the problem, which partitions the state-space of the problem into smaller sub-spaces. multiple strategies were developed to subside the problem of running the newly created sub-problems individually, such as using the symmetry of the created sub-circuits. In this paper we discussed the impact of FrozenQubits on sparser circuits with lower average connectivity degrees than the ones discussed in the original paper. We discussed the impact of FrozenQubits on sparser circuits on the number of CNOTs. Overall the method produces significantly worse results than with power-law type graphs leading to minor improvements in fidelity at the cost of computational overhead. 